# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Aug 10 2022 9:15AM,243846,10,PRF-34863,Bio-PRF,Released
1,Aug 10 2022 9:15AM,243846,10,PRF-34870,Bio-PRF,Released
2,Aug 10 2022 9:15AM,243846,10,PRF-34873,Bio-PRF,Released
3,Aug 10 2022 9:15AM,243846,10,PRF-34876,Bio-PRF,Released
4,Aug 10 2022 9:15AM,243846,10,PRF-34885,Bio-PRF,Released
5,Aug 10 2022 9:15AM,243846,10,PRF-34895,Bio-PRF,Released
6,Aug 10 2022 9:15AM,243846,10,PRF-34897,Bio-PRF,Released
7,Aug 10 2022 9:15AM,243846,10,PRF-34903,Bio-PRF,Released
8,Aug 10 2022 9:03AM,243845,12,HH-36333,Hush Hush,Released
9,Aug 10 2022 9:03AM,243845,12,HH-36336,Hush Hush,Executing


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
39,243841,Released,1
40,243842,Released,1
41,243845,Executing,2
42,243845,Released,4
43,243846,Released,8


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
243839,NaN,NaN,58.0
243841,NaN,NaN,1.0
243842,NaN,NaN,1.0
243845,NaN,2.0,4.0
243846,NaN,NaN,8.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
243723,70.0,1.0,37.0
243744,11.0,4.0,0.0
243755,0.0,0.0,8.0
243768,21.0,4.0,0.0
243769,0.0,0.0,9.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
243723,70,1,37
243744,11,4,0
243755,0,0,8
243768,21,4,0
243769,0,0,9


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,243723,70,1,37
1,243744,11,4,0
2,243755,0,0,8
3,243768,21,4,0
4,243769,0,0,9


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,243723,70,1,37
1,243744,11,4,
2,243755,,,8
3,243768,21,4,
4,243769,,,9


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Aug 10 2022 9:15AM,243846,10,Bio-PRF
8,Aug 10 2022 9:03AM,243845,12,Hush Hush
14,Aug 10 2022 8:30AM,243842,16,"SHL Pharma, LLC"
15,Aug 10 2022 8:30AM,243824,10,"Methapharm, Inc."
18,Aug 10 2022 8:27AM,243841,10,"Methapharm, Inc."
19,Aug 10 2022 8:25AM,243839,10,ISDIN Corporation
77,Aug 10 2022 8:13AM,243837,10,ISDIN Corporation
87,Aug 10 2022 8:11AM,243836,10,Beach Products Inc
109,Aug 10 2022 8:08AM,243835,10,Emerginnova
118,Aug 10 2022 7:54AM,243833,19,"AdvaGen Pharma, Ltd"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Aug 10 2022 9:15AM,243846,10,Bio-PRF,,,8
1,Aug 10 2022 9:03AM,243845,12,Hush Hush,,2,4
2,Aug 10 2022 8:30AM,243842,16,"SHL Pharma, LLC",,,1
3,Aug 10 2022 8:30AM,243824,10,"Methapharm, Inc.",,,3
4,Aug 10 2022 8:27AM,243841,10,"Methapharm, Inc.",,,1
5,Aug 10 2022 8:25AM,243839,10,ISDIN Corporation,,,58
6,Aug 10 2022 8:13AM,243837,10,ISDIN Corporation,,,10
7,Aug 10 2022 8:11AM,243836,10,Beach Products Inc,,,22
8,Aug 10 2022 8:08AM,243835,10,Emerginnova,,,9
9,Aug 10 2022 7:54AM,243833,19,"AdvaGen Pharma, Ltd",,,16


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 10 2022 9:15AM,243846,10,Bio-PRF,8,,
1,Aug 10 2022 9:03AM,243845,12,Hush Hush,4,2,
2,Aug 10 2022 8:30AM,243842,16,"SHL Pharma, LLC",1,,
3,Aug 10 2022 8:30AM,243824,10,"Methapharm, Inc.",3,,
4,Aug 10 2022 8:27AM,243841,10,"Methapharm, Inc.",1,,
5,Aug 10 2022 8:25AM,243839,10,ISDIN Corporation,58,,
6,Aug 10 2022 8:13AM,243837,10,ISDIN Corporation,10,,
7,Aug 10 2022 8:11AM,243836,10,Beach Products Inc,22,,
8,Aug 10 2022 8:08AM,243835,10,Emerginnova,9,,
9,Aug 10 2022 7:54AM,243833,19,"AdvaGen Pharma, Ltd",16,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 10 2022 9:15AM,243846,10,Bio-PRF,8,,
1,Aug 10 2022 9:03AM,243845,12,Hush Hush,4,2,
2,Aug 10 2022 8:30AM,243842,16,"SHL Pharma, LLC",1,,
3,Aug 10 2022 8:30AM,243824,10,"Methapharm, Inc.",3,,
4,Aug 10 2022 8:27AM,243841,10,"Methapharm, Inc.",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 10 2022 9:15AM,243846,10,Bio-PRF,8.0,NaN,NaN
1,Aug 10 2022 9:03AM,243845,12,Hush Hush,4.0,2.0,NaN
2,Aug 10 2022 8:30AM,243842,16,"SHL Pharma, LLC",1.0,NaN,NaN
3,Aug 10 2022 8:30AM,243824,10,"Methapharm, Inc.",3.0,NaN,NaN
4,Aug 10 2022 8:27AM,243841,10,"Methapharm, Inc.",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 10 2022 9:15AM,243846,10,Bio-PRF,8.0,0.0,0.0
1,Aug 10 2022 9:03AM,243845,12,Hush Hush,4.0,2.0,0.0
2,Aug 10 2022 8:30AM,243842,16,"SHL Pharma, LLC",1.0,0.0,0.0
3,Aug 10 2022 8:30AM,243824,10,"Methapharm, Inc.",3.0,0.0,0.0
4,Aug 10 2022 8:27AM,243841,10,"Methapharm, Inc.",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2438232,123.0,1.0,8.0
102,243818,1.0,0.0,0.0
12,487628,4.0,3.0,0.0
15,487478,45.0,1.0,70.0
16,243842,1.0,0.0,0.0
19,731396,22.0,10.0,29.0
20,1462697,70.0,54.0,16.0
21,1219140,5.0,0.0,0.0
22,243823,0.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2438232,123.0,1.0,8.0
1,102,243818,1.0,0.0,0.0
2,12,487628,4.0,3.0,0.0
3,15,487478,45.0,1.0,70.0
4,16,243842,1.0,0.0,0.0
5,19,731396,22.0,10.0,29.0
6,20,1462697,70.0,54.0,16.0
7,21,1219140,5.0,0.0,0.0
8,22,243823,0.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,123.0,1.0,8.0
1,102,1.0,0.0,0.0
2,12,4.0,3.0,0.0
3,15,45.0,1.0,70.0
4,16,1.0,0.0,0.0
5,19,22.0,10.0,29.0
6,20,70.0,54.0,16.0
7,21,5.0,0.0,0.0
8,22,0.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,123.0
1,102,Released,1.0
2,12,Released,4.0
3,15,Released,45.0
4,16,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,102,12,15,16,19,20,21,22
Status,,,,,,,,,
Completed,8.0,0.0,0.0,70.0,0.0,29.0,16.0,0.0,0.0
Executing,1.0,0.0,3.0,1.0,0.0,10.0,54.0,0.0,1.0
Released,123.0,1.0,4.0,45.0,1.0,22.0,70.0,5.0,0.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,102,12,15,16,19,20,21,22
0,Completed,8.0,0.0,0.0,70.0,0.0,29.0,16.0,0.0,0.0
1,Executing,1.0,0.0,3.0,1.0,0.0,10.0,54.0,0.0,1.0
2,Released,123.0,1.0,4.0,45.0,1.0,22.0,70.0,5.0,0.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,102,12,15,16,19,20,21,22
0,Completed,8.0,0.0,0.0,70.0,0.0,29.0,16.0,0.0,0.0
1,Executing,1.0,0.0,3.0,1.0,0.0,10.0,54.0,0.0,1.0
2,Released,123.0,1.0,4.0,45.0,1.0,22.0,70.0,5.0,0.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()